In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

from astropy.io import fits, ascii
from astropy import wcs
from astropy.coordinates import SkyCoord
from astropy import units as u

from IPython.display import Image
from skimage.io import imread,imsave

from toasty import toast, cartesian_sampler, normalizer

from collections import namedtuple
from itertools import product

import panstarrsTrig as trig 
import ps1skycell_toast as pssc

import time
import os
import sys

import warnings
warnings.filterwarnings("ignore", ".* is a low contrast image")

In [ ]:
def minmax(arr):
    return [min(arr),max(arr)]

In [ ]:
def M101Sampler():
    
    def vec2Pix(raArr,decArr):
        
        tile = np.zeros(raArr.shape)
        
        raArr[raArr < 0] += 2 * np.pi # making all ras 0 - 2pi
        
        raCorners = np.array([raArr[0][0],raArr[0][-1],raArr[-1][0],raArr[-1][-1]])
        decCorners = np.array([decArr[0][0],decArr[0][-1],decArr[-1][0],decArr[-1][-1]])       
        minRa = min(raCorners)
        maxRa = max(raCorners)
        minDec = min(decCorners)
        maxDec = max(decCorners)

        
        #   THIS IS ALL TEMPORARY 
        m101Pos = SkyCoord(210.79995, 54.34410, frame='icrs', unit='deg')
        if ((m101Pos.ra.rad >= minRa) and (m101Pos.ra.rad <= maxRa)):
            if ((m101Pos.dec.rad >= minDec) and (m101Pos.dec.rad <= maxDec)):
   
                fitsFile = fits.open('rings.v3.skycell.2381.053.stk.r.unconv.fits')
                rfWcs = wcs.WCS(fitsFile[1].header)
                footprint=np.array([SkyCoord(*x,frame='icrs',unit='deg') for x in rfWcs.calc_footprint(fitsFile[1].header) ])
                imgData = fitsFile[1].data
                fitsFile.close()
                
                fpMinRa = min(x.ra.rad for x in footprint)#(min(footprint[:,0])*u.deg).to(u.rad)
                fpMaxRa = max(x.ra.rad for x in footprint)#(max(footprint[:,0])*u.deg).to(u.rad)
                fpMinDec = min(x.dec.rad for x in footprint)#(min(footprint[:,1])*u.deg).to(u.rad)
                fpMaxDec = max(x.dec.rad for x in footprint)#(max(footprint[:,1])*u.deg).to(u.rad)

                print("woot")
                fpMask = (raArr <= fpMaxRa)  & (raArr >= fpMinRa) & (decArr <= fpMaxDec) & (decArr >= fpMinDec)
                print(np.sum(fpMask))
                print(fpMinRa,fpMaxRa)
                print(fpMinDec,fpMaxDec)
                print()
                
                ny, nx = imgData.shape[0:2]    
                
                tile[fpMask] = imgData[(ny*(decArr - fpMinDec)/(fpMaxDec - fpMinDec)).astype(int)[fpMask],
                                       (nx*(1-(raArr - fpMinRa)/(fpMaxRa - fpMinRa))).astype(int)[fpMask]]
                                
        # ^ THIS IS ALL TEMPORARY ^

        
        
        return tile
    
    return vec2Pix

In [ ]:
sampler = normalizer(M101Sampler(),-3,10)
output = 'toasts/ps1_M101'
depth = 8  
toast(sampler, depth, output, ra_range=[205,215],dec_range=[50,60])

In [ ]:
sampler = 'toasts/ps1_bottom'
output = 'toasts/ps1_bottom'
depth = 4 
toast(sampler, depth, output)

In [2]:
Data = namedtuple('Data','img cnt')

class fitsCache:
    """"Caching for fitfile image data"""
    def __init__(self,maxItems):
        self.cache = {}
        self.count = 0
        self.maxItms = maxItems
        
    def remove(self,filename):
        if filename not in self.cache:
            return
        del(self.cache[filename])
    
    def oldest(self):
        return sorted([(y.cnt,x) for x,y in seltf.cache.items()])[0][1]
    
    def add(self,filename):
        if len(self.cache) == self.maxItms:
            self.remove(self.oldest())
        try:
            fitsfile = fits.open(filename)      
            self.cache[filename] = Data(img=fitsfile[1].data,cnt=self.count)
            fitsfile.close()
        except:
            self.cache[filename] = Data(img=None,cnt=self.count)
        self.count += 1

    def get(self,filename):
        if filename not in self.cache:
            self.add(filename)
        return self.cache[filename].img 
    

In [3]:
psCache = fitsCache(10)

In [4]:
def panstarrsSampler():
    
    psCells = ascii.read("filter_r_rings.rpt")
    psSC2FileLoc = np.full((max(psCells['SCn']) + 1,max(psCells['SCm']) + 1),"",dtype='<U168')
    #psSC2FileLoc[psCells['SCn'],psCells['SCm']] = psCells['fileNPath']
    psSC2FileLoc[2404][5]  = "RINGS.V3.skycell.2404.005.stk.4032678.unconv.fits"
    psSC2FileLoc[2381][62] = "RINGS.V3.skycell.2381.062.stk.3906396.unconv.fits"
    psSC2FileLoc[2381][63] = "RINGS.V3.skycell.2381.063.stk.3906397.unconv.fits"
    psSC2FileLoc[2381][64] = "RINGS.V3.skycell.2381.064.stk.3906398.unconv.fits"
    psSC2FileLoc[2381][52] = "RINGS.V3.skycell.2381.052.stk.3906388.unconv.fits"
    psSC2FileLoc[2381][53] = "rings.v3.skycell.2381.053.stk.r.unconv.fits" # M101
    #psSC2FileLoc[2381][53] = "notanactualfile.fits"
    psSC2FileLoc[2381][54] = "RINGS.V3.skycell.2381.054.stk.3906390.unconv.fits"
    psSC2FileLoc[2381][42] = "RINGS.V3.skycell.2381.042.stk.3906380.unconv.fits"
    psSC2FileLoc[2381][43] = "RINGS.V3.skycell.2381.043.stk.3906381.unconv.fits"
    psSC2FileLoc[2381][44] = "RINGS.V3.skycell.2381.044.stk.3906382.unconv.fits"
    
    psSC2FileLoc[904][83] = "rings.v3.skycell.0904.083.stk.r.unconv.fits"
                        
    def vec2Pix(raArr,decArr):
        
        global psCache
        
        # Getting info about skycell and pixel location for given ra/decs
        pixelInfoArray = pssc.findskycell(raArr, decArr)
            
        # Getting the file paths and creating a unique list of them
        fileLocByPix = psSC2FileLoc[pixelInfoArray['projcell'],pixelInfoArray['subcell']] 
        filePths = {}
        for line in fileLocByPix:
            for filename in line:
                filePths[filename] = filename
        filePths = list(filePths.keys())
           
        tile = np.zeros(raArr.shape) # this should be filled with whatever we want to signal "no data" 
                                     # I am currently using zero
        #tile = np.full(raArr.shape,10)
        
        if (len(filePths) == 1) and (not filePths[0]):
            return None
        
        for dataFle in filePths:
            if dataFle == '':
                #print("File not available")
                continue
            
            #print("File location:",dataFle)
            
            # getting the image data (will be rplaced by cache class)
            #fitsFile = fits.open(dataFle)
            #imgData = fitsFile[1].data
            #fitsFile.close()   
            imgData = psCache.get(dataFle)
            
            # file did not have any associated image data, probably the file was not found on disc
            if imgData == None: 
                continue
                
            # getting the pixels we want out of this file
            pix2Fill = (fileLocByPix == dataFle)
            #print("Number of pixels to be filled:",np.sum(pix2Fill))
            #tile[pix2Fill] = imgData[pixIndY[pix2Fill],pixIndX[pix2Fill]]   
            tile[pix2Fill] = imgData[pixelInfoArray['y'][pix2Fill],pixelInfoArray['x'][pix2Fill]] 
        
            tile[np.isnan(tile)] = 10 # making nan values appear as white
        
        return tile   
        
    return vec2Pix

In [ ]:
start = time.time()
sampler = normalizer(panstarrsSampler(),-3.4,9.7, bias=.61428, contrast=1.81632)
output = 'toasts/ps1_M101_new'
depth = 8  
toast(sampler, depth, output, ra_range=[210.3,211],dec_range=[54,55.6])
end = time.time()

In [5]:
start = time.time()
sampler = normalizer(panstarrsSampler(),-2.267,9.001,scaling='sinh',bias=.22, contrast=3.723)
output = 'toasts/ps1'
depth = 12  
#toast(sampler, depth, output, ra_range=[210.589,211],dec_range=[54.26,54.45])
toast(sampler, depth, output, ra_range=[101,101.6],dec_range=[-16.9,-16.5],restart=True)
end = time.time()

Exception ignored in: <bound method FITS_rec.__del__ of FITS_rec([(array([101,   0], dtype=int32),),
       (array([9320,  101], dtype=int32),),
       (array([9978, 9421], dtype=int32),), ...,
       (array([    9211, 63724159], dtype=int32),),
       (array([    9410, 63733370], dtype=int32),),
       (array([     101, 63742780], dtype=int32),)], 
      dtype=(numpy.record, [('COMPRESSED_DATA', '>i4', (2,))]))>
Traceback (most recent call last):
  File "/Users/cbrasseur/anaconda/envs/toasty/lib/python3.5/site-packages/astropy/io/fits/fitsrec.py", line 640, in __del__
    del self._coldefs
  File "/Users/cbrasseur/anaconda/envs/toasty/lib/python3.5/site-packages/astropy/io/fits/fitsrec.py", line 637, in _coldefs
    raise AttributeError(exc.args[0])
AttributeError: _coldefs
/Users/cbrasseur/anaconda/envs/toasty/lib/python3.5/site-packages/ipykernel/__main__.py:64: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
/Users/cbrasseur/anacon

In [ ]:
end-start

In [ ]:
start = time.time()
sensitivity = 1000
sineArray = np.empty(360*sensitivity)
for angle in range(360*sensitivity):
    sineArray[angle] = np.sin((angle / sensitivity) * (np.pi / 180))

np.savetxt("sineLUT.001.fullCircle.txt",sineArray)
end = time.time()
print(end-start)

In [ ]:
sineArray = np.loadtxt("sineLUT.001.txt")
# plotting sine
plt.plot(np.array(range(len(sineArray)))/1000,sineArray,'b')
plt.plot(np.array(range(len(sineArray)))/1000 + 90,sineArray[::-1],'b')
plt.plot(np.array(range(len(sineArray)))/1000 + 180,sineArray*-1,'b')
plt.plot(np.array(range(len(sineArray)))/1000 + 270,sineArray[::-1]*-1,'b')
# plotting cosine
plt.plot(np.array(range(len(sineArray)))/1000,sineArray[::-1],'g')
plt.plot(np.array(range(len(sineArray)))/1000 + 90,sineArray*-1,'g')
plt.plot(np.array(range(len(sineArray)))/1000 + 180,sineArray[::-1]*-1,'g')
plt.plot(np.array(range(len(sineArray)))/1000 + 270,sineArray,'g')
# the axes
plt.axhline(0,color='k')
plt.show()

In [60]:
def colorize(depth,rDir,gDir,bDir,outDir,txrange,tyrange):
    for tx,ty in product(range(*txrange),range(*tyrange)):
        pth = '/' + str(depth) + '/' + str(ty) + '/' + str(ty) + '_' + str(tx) + '.png'
        if not (os.path.isfile(rDir + pth) and os.path.isfile(gDir + pth) and os.path.isfile(bDir + pth)):
            continue
        
        r = imread(rDir+pth)
        g = imread(gDir+pth)
        b = imread(bDir+pth)
       
        b = (b * 0.8).astype(np.uint8)
        #r = (r * 0.9).astype(np.uint8)
        #print(np.min(r),np.max(r))
        #print(np.min(g),np.max(g))
        #print(np.min(b),np.max(b))
        #print()
            
        rgb = np.dstack((r,g,b))

        direc, _ = os.path.split(outDir+pth)
        if not os.path.exists(direc):
            os.makedirs(direc)
        imsave(outDir+pth, rgb)

In [61]:
astroViewPath = '/Users/cbrasseur/Documents/Portal/Mashup/Clients/AstroView/ps1'

# M51
colorize(12,'colorizing/ps_r','colorizing/ps_g','colorizing/ps_i',astroViewPath,[1240,1244],[2356,2360])

# M1
colorize(12,'colorizing/ps_r','colorizing/ps_g','colorizing/ps_i',astroViewPath,[2172,2176],[608,616])

# Sirius
colorize(12,'colorizing/ps_r','colorizing/ps_g','colorizing/ps_i',astroViewPath,[1444,1448],[232,236])

# M101
colorize(12,'colorizing/ps_r','colorizing/ps_g','colorizing/ps_i',astroViewPath,[1400,1404],[2412,2416])

# 281.27063 -7.33885
colorize(12,'colorizing/ps_r','colorizing/ps_g','colorizing/ps_i',astroViewPath,[2460,2464],[3844,3848])